# CVD BB Pullback Strategy Backtest Demo

This notebook demonstrates how to backtest the CVD Bollinger Band Pullback strategy using parquet data files.

In [40]:
import sys
import pandas as pd
import numpy as np
from pathlib import Path

# Add src to path
sys.path.append('../../src')

from strategies.implementations.cvd_bb_pullback import CVDBBPullbackStrategy
from bt_engine.vectorbt_engine import VectorBTEngine
from config.settings import settings

In [41]:
data_path = Path('../../data/raw/market_data/intraday/okx_btc_usdt_perp_1m_2021_10_27-2024_10_26.parquet')
data = pd.read_parquet(data_path)

print(f"Data shape: {data.shape}")
print(f"Date range: {data.index.min()} to {data.index.max()}")
print(f"Columns: {list(data.columns)}")
data.head()

Data shape: (1576800, 6)
Date range: 0 to 1576799
Columns: ['datetime', 'open', 'high', 'low', 'close', 'volume']


,datetime,open,high,low,close,volume
0,2024-10-26 20:01:00,67138.296875,67162.296875,67136.000000,67154.898438,3312.3
1,2024-10-26 20:00:00,67178.500000,67178.500000,67138.296875,67138.296875,2229.7
2,2024-10-26 19:59:00,67163.500000,67178.500000,67158.101562,67178.500000,1873.5
3,2024-10-26 19:58:00,67184.703125,67184.703125,67163.500000,67163.500000,1244.6
4,2024-10-26 19:57:00,67181.703125,67184.898438,67181.703125,67184.601562,504.4


In [42]:
strategy = CVDBBPullbackStrategy()
engine = VectorBTEngine(
    initial_cash=1000,
    fee_pct=0.05,
    frequency='1min'
)

print(f"Strategy: {strategy.name}")
print(f"Default parameters: {strategy.params}")

Strategy: CVDBBPullbackStrategy
Default parameters: {'bbands_length': 30, 'bbands_stddev': 2.0, 'cvd_length': 50, 'atr_length': 14, 'sl_coef': 2.5, 'tpsl_ratio': 2.0}


In [43]:
param_ranges_small = {
    'bbands_length': [20, 30, 40],
    'bbands_stddev': [2.0, 2.5],
    'cvd_length': [30],
    'atr_length': [14],
    'sl_coef': [2.0],
    'tpsl_ratio': [2.5]
}

print(f"Parameter combinations: {np.prod([len(v) for v in param_ranges_small.values()])}")

Parameter combinations: 6


In [44]:
results = engine.simulate_portfolios(
    strategy=strategy,
    data=data,
    param_dict=param_ranges_small,
    ticker="BTC-USDT-PERP",
    sizing_method="Risk percent",
    risk_pct=1.0,
    exchange_broker="okx",
    date_range="2021_2024",
    save_results=False
)

print(f"Backtest completed! Results shape: {results.shape}")

Starting CVDBBPullbackStrategy Backtesting...
Total parameter combinations: 6
Processing in indicator batches of 500


Backtesting Progress: 100%|██████████| 6/6 [02:08<00:00, 21.39s/it]

All batches processed. Total combinations: 6
Backtest completed! Results shape: (6, 25)


In [45]:
top_results = results.nlargest(10, 'sharpe_ratio')

print("Top 10 parameter combinations by Sharpe ratio:")
display(top_results[['CVDBB_bbands_length', 'CVDBB_bbands_stddev', 'CVDBB_cvd_length', 
                    'total_return_pct', 'sharpe_ratio', 'max_drawdown_pct', 
                    'total_trades', 'win_rate_pct']])

Top 10 parameter combinations by Sharpe ratio:


,CVDBB_bbands_length,CVDBB_bbands_stddev,CVDBB_cvd_length,total_return_pct,sharpe_ratio,max_drawdown_pct,total_trades,win_rate_pct
5,40,2.5,30,-99.980761,-8.333888,99.980841,5227,28.582361
3,30,2.5,30,-99.981630,-9.616887,99.981630,4316,26.227989
1,20,2.5,30,-99.981997,-9.729706,99.981997,4349,26.488848
2,30,2.0,30,-99.981967,-10.154205,99.982106,3432,25.932401
0,20,2.0,30,-99.982164,-10.337559,99.982164,3241,25.794508
4,40,2.0,30,-99.982083,-10.550561,99.982087,3170,25.772871


In [46]:
import plotly.express as px

fig = px.scatter(
    results, 
    x='max_drawdown_pct', 
    y='total_return_pct',
    color='sharpe_ratio',
    size='total_trades',
    hover_data=['CVDBB_bbands_length', 'CVDBB_cvd_length', 'win_rate_pct'],
    title='Strategy Performance: Return vs Risk'
)
fig.show()